# Question 1
## Part 1.1 Implement an agent-based model 

You can use the following template or feel free to implement your own program. Please refer to the handout for details about parameters settings. 

In [210]:
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

#### Test check_success

In [211]:
#?? percent_mask
#?? percent_distancing

class Agent:
    '''
    Allowable statuses: (R) Recovered
                        (S) Susceptible to Infection
                        (I) Infected
                        (Q) Quarantined
                        
    Asymptomatic:  True if agent is asymptomatic when infected
    Distancing: True if agent is physically distancing
    Mask: True if agent is wearing a mask
    '''
    statuses = ['R', 'S', 'I', 'Q'] # allowable statuses
    days_infected = 0    # How long agent has been infected, if currently infected
    
    def __init__(self, 
                 position, 
                 status,  
                 mask, 
                 distancing):
        '''
        Initialize an agent.
        '''
        # position (x,y)
        self.position = position

        # status
        if status in self.statuses:
            self.status = status
        else:
            raise ValueError('status: ' + status + ' is not valid.')

        # possibly asymptomatic if infected
        if status == 'I':
            if self.is_asymptomatic():
                 self.asymptomatic = True
            else:
                self.asymptomatic = False
        else:
            self.asymptomatic = False 
                            
        self.mask = mask
        self.distancing = distancing 
        
    
    def is_event(self, num_event, num_outcomes):
        '''
        If an event has a 25% chance, num_event = 1 and num_outcomes = 4
        Returns true if event is selected from outcomes, False otherwise.
        '''
        return random.randint(num_event,num_outcomes) == num_event
    
    
    def is_infected(self, adjacent_agents):
        '''
        Returns True if the agent is now infected, given whether
        an adjacent agent is infected or masked and whether the 
        agent is masked. False otherwise.
        
        Check is performed for each agent adjacent to this agent. 
        ''' 
        infected = False
        
        # perform check for all adjacent agents
        for adjacent in adjacent_agents:
        
            # chance of infection if an adjacent agent is infected
            if adjacent.status == 'I':

                # infection probability depends on mask status of both
                if self.mask and adjacent.mask: 

                    if self.is_event(1,10000): # 0.01% chance
                        infected = True

                elif self.mask or adjacent.mask: 

                    if self.is_event(1,100):  # 1% chance
                        infected = True

                else:  

                    if self.is_event(1,4):   # 25% chance
                        infected = True
            
        return infected
            
    
    def is_asymptomatic(self):
        '''
        Returns True if agent is asymptomatic, False otherwise.
        '''
        return self.is_event(1,5) # 20% chance of being asymptomatic
        
    
    def will_quarantine(self):
        '''       
        Returns True if the agent has been infected for 
        at least 3 days and is symptomatic, False otherwise.
        '''       
        return self.days_infected > 2 and not self.asymptomatic

        
    
    def infection_over(self):
        '''
        Returns True if the agent has been infected for at
        least 15 days, False otherwise
        '''
        return self.days_infected > 14
                
    
    def update_status(self, adjacent_agents):
        '''
        Checks whether modification to the agent's status 
        is needed, and updates the status as necessary.
        '''
        
        if self.status == 'S': # not yet infected
            
            if self.is_infected(adjacent_agents):
                self.status = 'I'
                
        elif self.status == 'I': # already infected
            
            if self.will_quarantine():
                self.status = 'Q'
        
            if self.infection_over():
                self.status = 'R'
                
        elif self.status == 'Q': # already in quarantine
            
            if self.infection_over():
                self.status = 'R'
            
            
    def update_agent(self, new_position, adjacent_agents):
        '''
        Updates all agent fields based on given position and 
        adjacent agents.
        '''
        
        # update position
        self.position = new_position
        
        # update days infected if already infected
        if self.status == 'I' or self.status == 'Q':
            self.days_infected += 1 # day passed since infection started            
        
        # update status
        status_before = self.status
        self.update_status(adjacent_agents)
        
         # if now infected, determine if asymptomatic
        if status_before != 'I' and self.status == 'I':
            self.asymptomatic = self.is_asymptomatic()
            
        elif status_before != 'R' and self.status == 'R':
            
            # reset metrics related to infection
            self.days_infected = 0
            self.asymptomatic = False
            
    
    def __str__(self):  
        return  'Agent{'+'position:'+ str(self.position)+', status:'+ str(self.status)+', mask:'+ str(self.mask)+', distancing:'+ str(self.distancing)+', days_infected:'+ str(self.days_infected)+', asymptomatic:'+ str(self.asymptomatic)+'}'      
    
    

#### Test Agent class

In [212]:
print("Test constructor")
a1 = Agent((1,1), 'I', mask=True, distancing=True)

print(a1)

num = 100000
counter = 0
for i in range(num):
    a1 = Agent((1,1), 'I', mask=True, distancing=True)
    if a1.asymptomatic == True:
        counter += 1
assert round(counter/num,2) == 0.20


Test constructor
Agent{position:(1, 1), status:I, mask:True, distancing:True, days_infected:0, asymptomatic:False}


In [213]:
print('Test is_event()')

num = 100000
counter = 0
for i in range(num):
    if is_event(1,10000):
        counter += 1
assert round(counter/num,4) == 0.0001

counter = 0
for i in range(num):
    if is_event(1,100):
        counter += 1
assert round(counter/num,2) == 0.01

counter = 0
for i in range(num):
    if is_event(1,4):
        counter += 1
assert round(counter/num,2) == 0.25

Test is_event()


In [214]:
print('Test is_asymptomatic()')

counter = 0
for i in range(num):
    if a1.is_asymptomatic():
        counter += 1
assert round(counter/num,2) == 0.20

Test is_asymptomatic()


In [215]:
print('Test will_quarantine()')

a2 = Agent((0,1), 'I', mask=True, distancing=True)

# possibility of quarantine when symptomatic
a2.asymptomatic = False

a2.days_infected = 2
assert a2.will_quarantine() == False

a2.days_infected = 3
assert a2.will_quarantine() == True

a2.days_infected = 4
assert a2.will_quarantine() == True


# when asymptomatic, will not quarantine
a2.asymptomatic = True
a2.days_infected = 2
assert a2.will_quarantine() == False

a2.days_infected = 3
assert a2.will_quarantine() == False

a2.days_infected = 4
assert a2.will_quarantine() == False


Test will_quarantine()


In [216]:
print('Test infection_over()')
a3 = Agent((0,1), 'I', mask=True, distancing=True)
a3.days_infected = 13
assert a3.infection_over() == False

a3.days_infected = 14
assert a3.infection_over() == False

a3.days_infected = 15
assert a3.infection_over() == True


Test infection_over()


In [217]:
print('Test is_infected()')
a1 = Agent((1,1), 'S', mask=True, distancing=True)
a2 = Agent((0,1), 'I', mask=True, distancing=True)

# test 1 adjacent, 1 infected, both masked
a1.mask = True
a2.mask = True
num = 100000
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,4) == 0.0001

# test 1 adjacent, 1 infected, only agent masked
a1.mask = True
a2.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 1 adjacent, 1 infected, only adjacent masked
a1.mask = False
a2.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 1 adjacent, 1 infected, no masks
a1.mask = False
a2.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2]):
        counter += 1
assert round(counter/num,2) == 0.25

Test is_infected()


In [218]:
print('Test is_infected()')
a3 = Agent((1,2), 'R', mask=True, distancing=True)

# test 2 adjacent, 1 infected, all masked
a1.mask = True
a2.mask = True
a3.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,4) == 0.0001

# test 2 adjacent, 1 infected, only agent masked
a1.mask = True
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,2) == 0.01

# test 2 adjacent, 1 infected, no one masked
a2.mask = False
a3.mask = False
a1.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
assert round(counter/num,2) == 0.25

Test is_infected()


In [219]:
print('Test is_infected()')
# test 2 adjacent, 2 infected, all masked
a3.status = 'I'
a1.mask = True
a2.mask = True
a3.mask = True
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,4))
assert round(counter/num,4) > 0.0001

# test 2 adjacent, 2 infected, agent masked
a3.status = 'I'
a1.mask = True
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) > 0.01

# test 2 adjacent, 2 infected, agent masked, one adjacent masked
a3.status = 'I'
a1.mask = True
a2.mask = True
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) >= 0.01

# test 2 adjacent, 2 infected, no one masked
a3.status = 'I'
a1.mask = False
a2.mask = False
a3.mask = False
counter = 0
for i in range(num):
    if a1.is_infected([a2,a3]):
        counter += 1
print(round(counter/num,2))
assert round(counter/num,2) > 0.25

Test is_infected()
0.0003
0.02
0.01
0.44


In [220]:
print('Test update_status()')
a1 = Agent((1,1), 'R', mask=True, distancing=True)
a2 = Agent((0,1), 'I', mask=False, distancing=True)

# test R status
print('Test R status...')
a1.update_status([a2])
assert a1.status == 'R'

# test I status
print('Test I status...')

a1.status = 'I'
a1.days_infected = 0 # test not yet quarantined
a1.asymptomatic = True
a1.update_status([a2])  
assert a1.status == 'I'


a1.status = 'I'
a1.days_infected = 3 # test change to quarantined
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'Q'

a1.status = 'I'
a1.days_infected = 15 # test change to recovered
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'R'

# test Q status
print('Test Q status...')
a1.status = 'Q'
a1.days_infected = 15 # test change to recovered
a1.asymptomatic = False
a1.update_status([a2]) 
assert a1.status == 'R'

# test S status
print('Test S status...')
a1.status = 'S'
a1.days_infected = 0
a1.asymptomatic = False
num = 10000
counter = 0
for i in range(num):
    a1.status = 'S'
    a1.update_status([a2])
    if a1.status == 'I':
        counter += 1
assert round(counter/num,2) == 0.01

Test update_status()
Test R status...
Test I status...
Test Q status...
Test S status...


In [224]:
print('Test update_agent()')

a1 = Agent((1,1), 'R', mask=False, distancing=True)
a2 = Agent((0,1), 'I', mask=False, distancing=True)
a1.update_agent((2,2), [a2])

# check position
assert a1.position == (2,2)

# check days_infected
a1.status = 'I'
a1.update_agent((2,2), [a2])
assert a1.days_infected == 1

# check status if now recovered
# check metrics if now recovered
a1.status = 'I'
a1.days_infected = 14
a1.update_agent((2,2), [a2])
assert a1.status == 'R'
assert a1.days_infected == 0
assert a1.asymptomatic == False

# check status if now recovered
# check metrics if now recovered
a1.status = 'Q'
a1.days_infected = 14
a1.update_agent((2,2), [a2])
assert a1.status == 'R'
assert a1.days_infected == 0
assert a1.asymptomatic == False
    

# check asymptomatic if now infected
a1.status = 'S'
a1.days_infected = 0
a1.asymptomatic = False
a1.update_agent((2,2), [a2])

num = 100000
counter = 0
for i in range(num):
    a1.status = 'S'
    a1.asymptomatic = False
    a1.update_agent((2,2), [a2])
    if a1.asymptomatic == True:
        counter += 1
assert round(counter/num,2) == 0.05

Test update_agent()


In [222]:
print('All tests pass.')

All tests pass.


In [ ]:
class ABM:
    directions = [[-1,0], # left
                [1,0], # right
                [0,-1], # up
                [0,1], # down
                [-1,-1], # left & up
                [-1,1], # left & down
                [1,-1], # right & up
                [1,1]] # right & down

    statusColors = {'R': 'r', 'S': 'b', 'I': 'g', 'Q': 'k', 'D': 'm'}

    def __init__(self, n, m, num_infected, percent_distancing, percent_mask, percent_vaccinated = 0):
        """
        Initialize the world and agents in the world. 

        Parameters:
          n: size of the world
          m: number of agents 

        """

    def moveAgent(self, agent):
        """
        Move an agent and update status based on conditions. 

        """

    def agent_dies(self,agent):
        '''
        Check if agent dies due to infection.
        '''    


    def remove_agent(self,agent):
        '''
        Remove agent from simulation if agent has died.
        '''     



    def addCounts(self):
        counts = {'R': self.num_recovered, 'D': self.num_dead, 'I': self.num_infected, 'S': self.num_susceptible, 'Q': self.num_quarantine}
        self.counts.append(counts)

    def runSimulation(self, num_steps):
        for t in range(num_steps):
            for agent in self.agents:
                self.moveAgent(agent)

                self.addCounts()

        return self.counts


    def runAndVisualizeSimulation(self, num_steps):
        """
        Helper function for de-bugging. 

        """

        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)

        for t in range(num_steps):
            ax.cla()

        for agent in self.agents:
            self.moveAgent(agent)
            c = self.statusColors[agent.status]
            m = 'o' if agent.is_distancing else 's'

            ax.scatter(agent.x, agent.y, c=c, marker=m)

        ax.grid(True)
        ax.set_xticks(list(range(self.n)))
        ax.set_yticks(list(range(self.n)))  

        display(fig)
        clear_output(wait=True)

        plt.pause(5)
        
        

In [ ]:
 def runAndPlotSimulations(num_simulations, num_steps, world_size, num_agents, num_infected, percent_distancing, percent_mask, percent_vaccinated):
    sim_counts = []
    for i in range(num_simulations):
      model = ABM(world_size, num_agents, num_infected, percent_distancing, percent_mask, percent_vaccinated)
      counts = model.runSimulation(num_steps)
      sim_counts.append(counts)


    statusColors = {'R': 'r', 'S': 'b', 'I': 'g', 'Q': 'k', 'D': 'm'}
    fig, ax = plt.subplots(figsize=(20, 10))
    x = list(range(num_steps))

    for status in ['R', 'S', 'I', 'Q', 'D']:
      avgs = []
      errs = []
      for j in range(num_steps):
        stats = np.array([counts[j][status] for counts in sim_counts])
        avgs.append(np.mean(stats))
        errs.append(np.std(stats))

      ax.errorbar(x,avgs,yerr=errs, fmt=statusColors[status], label=status)

    ax.legend(loc="upper left", bbox_to_anchor=(1, 0.5))
    plt.show()

## 1.2 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 

* 5 infected agents (I), the rest are susceptible (S)
* 0% of agents wear masks
* 0% of agents practice physical distancing

Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard error bars. 
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 


In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)

## 1.3 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 


*   5 infected agents (I), the rest are susceptible (S)
*   p% of agents wear masks
*   p% of agents practice physical distancing

Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard error bars. 
What value of p reduces the final value of R to roughly half of the final value of R you obtained for part 1.2?
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 

In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)

## 1.4 Run 10 simulations, each of which tracks 250 agents over 365 days in a 25 x 25 world with the following initial conditions: 


*   5 infected agents (I), p% of agents have been vaccinated (i.e., start in state R), the rest are susceptible (S)
*   0% of agents wear masks
*   0% of agents practice physical distancing


Keep track of the number of S, I,Q,R, and D agents over time in each simulation. Plot the averages of these trajectories with standard errors. 
What value of p reduces the peak number of active cases (I+Q) to roughly half of the final value of (I+Q) you obtained for part 1.2?
What is the final value of R?
What is the peak number of active cases (I+Q)? 
How many days does it take for the virus to go extinct (i.e. I+Q=0)? 

In [ ]:
# runAndPlotSimulations(10, 365, 25, 250, num_infected, percent_distancing, percent_mask, percent_vaccinated)